<a href="https://colab.research.google.com/github/bansal19/Deep_Learning_Bittensor/blob/main/network_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tabulate bittensor plotly wandb pandas

In [5]:
import wandb
import bittensor as bt
import pandas as pd
import plotly.express as px

# Check current metagraph state

In [6]:
s = bt.subtensor(network='test')


#

In [ ]:
# TODO: set your coldkeys here
my_coldkeys = ['<COLDKEY-1>','<COLDKEY-2>']


In [ ]:
# grab current metagraph
m = s.metagraph(1)

In [ ]:
# make a dataframe from current metagraph snapshot and tag your miners

meta_cols = ['I','stake','trust','validator_trust','C','R','E','dividends','last_update']

df_m = pd.DataFrame({k: getattr(m, k) for k in meta_cols})
df_m['uid'] = range(m.n.item())
df_m['hotkey'] = list(map(lambda a: a.hotkey, m.axons))
df_m['coldkey'] = list(map(lambda a: a.coldkey, m.axons))
df_m['ip'] = list(map(lambda a: a.ip, m.axons))
df_m['port'] = list(map(lambda a: a.port, m.axons))

df_m['my_miners'] = df_m.coldkey.isin(my_coldkeys)
df_m.sort_values(by=['stake'], inplace=True, ascending=False)
df_m

In [ ]:
# Get overview of your miners
df_m.loc[df_m['my_miners'] == True]

In [ ]:
fig = px.scatter(df_m.assign(
                    msize=1+3*df_m['my_miners'].astype(float)
                  ).sort_values('I', ascending=False).reset_index(), y='I',
                 color='my_miners', size='msize', opacity=0.5,
                 hover_name='uid', hover_data=['hotkey','coldkey'],
                 title=f'My miners ({df_m.my_miners.sum()}) at block {s.block}',
                 labels={'_index':'Ranking'},
                 width=800, height=600, template='plotly_white',
)
fig.update_traces(marker=dict(line=dict(width=0)))

# Live network analysis using weights and biases data

In [ ]:
api = wandb.Api()

In [ ]:
# TODO: select a wandb run from https://wandb.ai/opentensor-dev/alpha-validators/table?workspace=default

run_path = 'opentensor-dev/alpha-validators/30sg7cpc' # e.g. TAOSTATS live run
run = api.run(run_path)

# may take a while if the run is long
df = pd.DataFrame(list(run.scan_history()))
df._timestamp = df._timestamp.apply(pd.to_datetime, unit='s')
df


In [ ]:
list_cols = ['uids','completions','rewards','status_codes','status_messages','relevance_raw_reward','rouge_raw_reward','rouge_raw_penalty','timings']
scalar_cols = ['_step','_timestamp','reference','query','task','challenge']
# 'unroll' the batches of completions into a long dataframe
df_long = df[scalar_cols+list_cols].explode(list_cols)
df_long

In [ ]:
# look at overall status codes
df_long.status_codes.value_counts()


In [ ]:
# look at number of unique uids which responded with each status code
df_long.groupby('status_codes').uids.nunique()

In [ ]:
my_uids = df_m.loc[df_m['my_miners']==True,'uid'].tolist()
my_uids

In [ ]:
# look at status codes for my miners
df_long_my_miners = df_long.loc[df_long.uids.isin(my_uids)]
df_long_my_miners.groupby('uids').status_codes.value_counts().unstack().fillna(0)

In [ ]:
df_long['status'] = df_long['status_messages'].apply(lambda x: x.split(':')[0] if isinstance(x,str) else x)

In [ ]:
df_time =df_long.groupby(pd.Grouper(key='_timestamp', freq='1h')).status_codes.value_counts(normalize=True).rename('proportion').reset_index()
df_time

In [ ]:
px.line(df_time.assign(percent=df_time.proportion*100), x='_timestamp', y='percent', color='status_codes',
        width=800, height=600, template='plotly_white',
        hover_name='status_codes', hover_data=['status_codes','_timestamp','percent'],
        labels={'_timestamp':'', 'percent':'% of Requests', 'status_codes':'Status Code'},
        title='Status Codes for Validator over Time'
)

# Top Miners

In [ ]:
top_uids = df_long.groupby('uids').rewards.mean().sort_values(ascending=False)
top_uids.head(20)

In [ ]:
# Create a markdown file containing the most re which you can open and read later
num_top_uids = 5
num_completions = 10
with open('COMPLETIONS.md','w') as f:

    f.write(f'\n------\n')
    f.write(f'# Top {num_top_uids} UIDs\n')
    for uid in top_uids.index[:num_top_uids]:
        f.write(f'------\n')
        f.write(f'## UID {uid}\n')
        df_uid = df_long.loc[df_long.uids==uid]
        f.write(df_uid.status_messages.value_counts().to_markdown())
        f.write(f'\n')

        for idx, row in df_uid.loc[df_uid.rewards>0].tail(num_completions).iterrows():
            f.write(f'\n**Task** {row.task}')
            f.write(f'\n- Index {idx}')
            f.write(f'\n- Reward {row.rewards:.3f}')
            f.write(f'\n- Rouge {row.rouge_raw_reward:.3f}')
            f.write(f'\n- Relevance {row.relevance_raw_reward:.3f}')
            f.write(f'\n- Rouge Penalty {row.rouge_raw_penalty:.3f}')
            f.write(f'\n- Timing {row.timings:.3f}\n')
            f.write(f'\n**QUERY**\n{row.query}')
            f.write(f'\n**CHALLENGE**\n{row.challenge}')
            f.write(f'\n**REFERENCE**\n{row.reference}')
            f.write(f'\n**COMPLETION**\n{row.completions}')
            f.write(f'\n------\n')


In [ ]:
df_long['top']=df_long.uids.isin(top_uids.index[:20])
df_long['mine']=df_long.uids.isin(my_uids)


In [ ]:
sel = df_long.loc[df_long.rewards>0].melt(
        id_vars=['_timestamp','uids','top','mine','task'],
        value_vars=['relevance_raw_reward','rouge_raw_reward','rouge_raw_penalty','rewards'],
        var_name='metric',
        value_name='score'
        )

px.box(sel, x='score', facet_col='metric', facet_col_wrap=2,
        color='mine',
        category_orders={'top':[True,False]},
        title='Scores for my Miners',
       points=False,
        template='plotly_white', width=800, height=600, ).show()

In [ ]:
px.histogram(df_long.assign(failed=df_long.rewards==0),
             x='failed', color='mine', opacity=0.7,
             histnorm='percent', barmode='group',
             category_orders={'status_codes':sorted(df_long.status_codes.fillna(0).unique())},
             title='Failure Rates for my Miners',
             width=800, height=600, template='plotly_white',
)

In [ ]:
px.histogram(df_long.astype({'status_codes':str}),
             x='status_codes', color='mine', opacity=0.7,
             histnorm='percent', barmode='group',
             category_orders={'status_codes':sorted(df_long.status_codes.fillna(0).unique())},
             title='Status Codes for my Miners',
             width=800, height=600, template='plotly_white',
)